In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import tensorflow as tf
from sklearn.utils import shuffle

from tensorflow.keras.applications import ResNet50, DenseNet121
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.losses import *
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Prepare Original Dataset

In [ ]:
pd_train_images = np.load('/content/drive/MyDrive/Research/PPMI/PD-TRAIN-IMAGES.npy')
pd_test_images = np.load('/content/drive/MyDrive/Research/PPMI/PD-TEST-IMAGES.npy')
pd_train_labels = np.ones((pd_train_images.shape[0],1))
pd_test_labels = np.ones((pd_test_images.shape[0],1))

hc_train_images = np.load('/content/drive/MyDrive/Research/PPMI/HC-TRAIN-IMAGES.npy')
hc_test_images = np.load('/content/drive/MyDrive/Research/PPMI/HC-TEST-IMAGES.npy')
hc_train_labels = np.zeros((hc_train_images.shape[0],1))
hc_test_labels = np.zeros((hc_test_images.shape[0],1))

In [ ]:
X_train = np.concatenate((pd_train_images,hc_train_images))
X_test = np.concatenate((pd_test_images,hc_test_images))

y_train = np.concatenate((pd_train_labels,hc_train_labels))
y_test = np.concatenate((pd_test_labels,hc_test_labels))

In [ ]:
X_train,y_train = shuffle(X_train,y_train)
X_train.shape,y_train.shape

((1120, 128, 128, 1), (1120, 1))

## Add Augmented Images from Generative models

In [ ]:
pd_generated = np.load('/content/drive/MyDrive/Research/PPMI/GAN1-FT-PD.npy')
pd_test_labels = np.ones((pd_generated.shape[0],1))

hc_generated = np.load('/content/drive/MyDrive/Research/PPMI/GAN1-FT-HC.npy')
hc_labels = np.zeros((hc_generated.shape[0],1))

generated_images = np.concatenate((pd_generated,hc_generated))
generated_labels = np.concatenate((pd_test_labels,hc_labels))

X_train = np.concatenate((X_train,generated_images))
y_train = np.concatenate((y_train,generated_labels))

X_train,y_train = shuffle(X_train,y_train)
X_train,y_train = shuffle(X_train,y_train)

In [ ]:
X_train.shape,y_train.shape

((3120, 128, 128, 1), (3120, 1))

## Expand Channels

In [ ]:
X_train_before = X_train
X_test_before = X_test

X_train = np.zeros((X_train_before.shape[0],128,128,3))
X_test = np.zeros((X_test_before.shape[0],128,128,3))

for i in range(X_train_before.shape[0]):
  X_train[i,:,:,0] = X_train_before[i,:,:,0]
  X_train[i,:,:,1] = X_train_before[i,:,:,0]
  X_train[i,:,:,2] = X_train_before[i,:,:,0]

for i in range(X_test_before.shape[0]):
  X_test[i,:,:,0] = X_test_before[i,:,:,0]
  X_test[i,:,:,1] = X_test_before[i,:,:,0]
  X_test[i,:,:,2] = X_test_before[i,:,:,0]

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3120, 128, 128, 3), (832, 128, 128, 3), (3120, 1), (832, 1))

## Build and Train Model (ResNet)

In [ ]:
base_model = ResNet50(include_top = False, input_shape=(128,128,3))

In [ ]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

In [ ]:
def classification_model():
    inputs = Input((128,128,3))
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dense(64,activation='relu')(x)
    x = Dense(10,activation='relu')(x)
    x = Dense(1,activation='sigmoid')(x)
    model = Model(inputs,x)
    return model

In [ ]:
model = classification_model()
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d_5   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_15 (Dense)            (None, 64)                131136    
                                                                 
 dense_16 (Dense)            (None, 10)                650       
                                                                 
 dense_17 (Dense)            (None, 1)                 11        
                                                           

In [ ]:
model.compile(optimizer='Adam',loss=BinaryCrossentropy(),metrics =["accuracy"])

In [ ]:
gc.collect()

10302

In [ ]:
weight_saver = ModelCheckpoint('/content/Resnet.h5',
                            monitor= "val_loss",
                            save_best_only=True,
                            save_weights_only=True)
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=3,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)
callbacks = [weight_saver,lr_scheduler]
if os.path.exists('/content/Resnet.h5'):
    os.remove('/content/Resnet.h5')

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=20,validation_data=(X_test,y_test),callbacks=callbacks)

Epoch 1/20
98/98 [==============================] - 22s 150ms/step - loss: 0.1181 - accuracy: 0.9542 - val_loss: 172.4691 - val_accuracy: 0.3077 - lr: 0.0010
Epoch 2/20
98/98 [==============================] - 14s 141ms/step - loss: 0.0459 - accuracy: 0.9856 - val_loss: 2.6226 - val_accuracy: 0.3077 - lr: 0.0010
Epoch 3/20
98/98 [==============================] - 14s 138ms/step - loss: 0.0207 - accuracy: 0.9942 - val_loss: 2.7506 - val_accuracy: 0.3077 - lr: 0.0010
Epoch 4/20
98/98 [==============================] - 13s 138ms/step - loss: 0.0226 - accuracy: 0.9920 - val_loss: 5.8102 - val_accuracy: 0.3077 - lr: 0.0010
Epoch 5/20
98/98 [==============================] - 14s 138ms/step - loss: 0.0278 - accuracy: 0.9920 - val_loss: 1.9343 - val_accuracy: 0.3077 - lr: 0.0010
Epoch 6/20
98/98 [==============================] - 14s 140ms/step - loss: 0.0321 - accuracy: 0.9891 - val_loss: 1.2996 - val_accuracy: 0.6923 - lr: 0.0010
Epoch 7/20
98/98 [==============================] - 13s 136ms/

In [ ]:
model.load_weights('/content/Resnet.h5')

## Build and Train Model (DenseNet)

In [ ]:
base_model = DenseNet121(include_top = False, input_shape=(128,128,3))

29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1024) dtype=float32 (created by layer 'relu')>

In [ ]:
def classification_model():
    inputs = Input((128,128,3))
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dense(64,activation='relu')(x)
    x = Dense(10,activation='relu')(x)
    x = Dense(1,activation='sigmoid')(x)
    model = Model(inputs,x)
    return model

In [ ]:
model = classification_model()
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, 4, 4, 1024)        7037504   
                                                                 
 global_average_pooling2d_4   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_12 (Dense)            (None, 64)                65600     
                                                                 
 dense_13 (Dense)            (None, 10)                650       
                                                                 
 dense_14 (Dense)            (None, 1)                 11        
                                                           

In [ ]:
model.compile(optimizer='Adam',loss=BinaryCrossentropy(),metrics =["accuracy"])

In [ ]:
gc.collect()

17888

In [ ]:
weight_saver = ModelCheckpoint('/content/Densenet.h5',
                            monitor= "val_loss",
                            save_best_only=True,
                            save_weights_only=True)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=3,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0,
)
callbacks = [weight_saver,lr_scheduler]
if os.path.exists('/content/Densenet.h5'):
    os.remove('/content/Densenet.h5')

In [ ]:
model.fit(X_train,y_train,batch_size=32,epochs=20,validation_data=(X_test,y_test),callbacks=callbacks)

Epoch 1/20
98/98 [==============================] - 31s 182ms/step - loss: 0.1167 - accuracy: 0.9516 - val_loss: 8.3588 - val_accuracy: 0.5577 - lr: 0.0010
Epoch 2/20
98/98 [==============================] - 14s 146ms/step - loss: 0.0163 - accuracy: 0.9942 - val_loss: 2.4243 - val_accuracy: 0.7188 - lr: 0.0010
Epoch 3/20
98/98 [==============================] - 14s 138ms/step - loss: 0.0424 - accuracy: 0.9856 - val_loss: 15.9389 - val_accuracy: 0.3942 - lr: 0.0010
Epoch 4/20
98/98 [==============================] - 14s 142ms/step - loss: 0.0189 - accuracy: 0.9946 - val_loss: 3.2721 - val_accuracy: 0.6947 - lr: 0.0010
Epoch 5/20
98/98 [==============================] - 14s 146ms/step - loss: 0.0081 - accuracy: 0.9981 - val_loss: 0.9417 - val_accuracy: 0.8197 - lr: 0.0010
Epoch 6/20
98/98 [==============================] - 14s 145ms/step - loss: 9.4387e-04 - accuracy: 0.9997 - val_loss: 0.7707 - val_accuracy: 0.7692 - lr: 0.0010
Epoch 7/20
98/98 [==============================] - 13s 137

In [ ]:
model.load_weights('/content/Densenet.h5')

## Test Images

In [ ]:
y_pred = model.predict(X_test)

y_pred = (y_pred>0.5).astype(np.int32)

26/26 [==============================] - 1s 38ms/step


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
accuracy_score(y_test,y_pred), precision_score(y_test,y_pred), recall_score(y_test,y_pred), f1_score(y_test,y_pred)

(0.7596153846153846,
 0.8241379310344827,
 0.8298611111111112,
 0.8269896193771626)

In [ ]:
(0.796875, 0.8194662480376766, 0.90625, 0.8606760098928278)

In [ ]:
# resnet = (0.6923076923076923, 0.6923076923076923, 1.0, 0.8181818181818181)
#resnet + GAN1 = (0.6899038461538461,0.6929611650485437,0.9913194444444444,0.8157142857142856)
# resnet + GAN2 = ((0.7620192307692307,0.7755102040816326, 0.9236111111111112, 0.8431061806656102))
# resnet + Diffusion = (0.8497596153846154,0.9033989266547406,0.8767361111111112,0.8898678414096917)
# resnet + VAE = (0.8209134615384616, 0.8460291734197731, 0.90625, 0.8751047778709137)

# DenseNet = (0.7716346153846154,0.7789017341040463,0.9357638888888888,0.8501577287066246)
# DenseNet + GAN1 = (0.7633173076923077, 0.780627802690583, 0.933125, 0.8519277108433735)
# DenseNet + GAN2 = (0.7980769230769231, 0.8119266055045872, 0.921875, 0.8634146341463416)
# DenseNet + Diffusion = (0.9242788461538461,0.9621621621621622,0.9270833333333334,0.9442970822281168)
# DenseNet + VAE = (0.8401442307692307,0.8825561312607945,0.8871527777777778,0.8848484848484849)